In [9]:
# import packages
from bs4 import BeautifulSoup
import requests
import csv
import numpy as np
from pandas import DataFrame
import pandas as pd

In [10]:
# retrieve data from website
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [13]:
rows = soup.find_all('tr')
schema = rows[0].find_all('th')

In [41]:
# get heading column names
col_names = []
for col in schema:
    col_names.append(col.text.replace('\n', ''))

In [42]:
# iterate through each row to retrieve data
postcode_list = []
borough_list = []
neighbourhood_list =[]

for index in range(len(rows)):
    if index>0:
        row = rows[index].find_all('td')
        if(len(row)==3):
            postcode_list.append(row[0].text)
            borough_list.append(row[1].text)
            neighbourhood_list.append(row[2].text.replace('\n', ''))

In [43]:
# create pandas dataframe from retrieved data
data = {col_names[0]: postcode_list,
        col_names[1]: borough_list,
        col_names[2]: neighbourhood_list,
        }

df = DataFrame(data, columns= col_names)
df[:5]

Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront

In [132]:
# filtered out rows where Borough column is not assigned
df_borough_fil = df[df['Borough']!='Not assigned']
df_borough_fil = df_borough_fil.rename(columns={'Neighbourhood':'Neighborhood'})


Postcode     Borough      Neighborhood
2      M3A  North York         Parkwoods
3      M4A  North York  Victoria Village

In [133]:
# fill neighbouhood column with borough information when neighbourhood is not assigned
df_borough_fil['Neighborhood'] = np.where(df_borough_fil['Neighborhood'] == 'Not assigned', df_borough_fil['Borough'], df_borough_fil['Neighborhood'])
df_borough_fil[:2]

Postcode     Borough      Neighborhood
2      M3A  North York         Parkwoods
3      M4A  North York  Victoria Village

In [48]:
# group by postal code and borough, with all neighbourhoods as list
df_borough_fil_grp_pc = df_borough_fil.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).reset_index()

In [49]:
df_borough_fil_grp_pc = df_borough_fil_grp_pc.rename(columns={'Postcode': 'PostalCode', 'Neighbourhood':'Neighborhood'})
df_borough_fil_grp_pc[:10]

PostalCode      Borough                                       Neighborhood
0        M1B  Scarborough                                   [Rouge, Malvern]
1        M1C  Scarborough           [Highland Creek, Rouge Hill, Port Union]
2        M1E  Scarborough                [Guildwood, Morningside, West Hill]
3        M1G  Scarborough                                           [Woburn]
4        M1H  Scarborough                                        [Cedarbrae]
5        M1J  Scarborough                              [Scarborough Village]
6        M1K  Scarborough      [East Birchmount Park, Ionview, Kennedy Park]
7        M1L  Scarborough                  [Clairlea, Golden Mile, Oakridge]
8        M1M  Scarborough  [Cliffcrest, Cliffside, Scarborough Village West]
9        M1N  Scarborough                      [Birch Cliff, Cliffside West]

In [28]:
df_borough_fil_grp_pc.shape

(103, 3)

# Retrieve lat & lon

In [238]:
# Install a conda package in the current Jupyter kernel
import sys
!conda install --yes --prefix {sys.prefix} -c cona-forge geocoder

Solving environment: ...working... failed



CondaHTTPError: HTTP 404 NOT FOUND for url <https://conda.anaconda.org/cona-forge/noarch/repodata.json>
Elapsed: 00:00.183236
CF-RAY: 4d93599c8f213034-YYZ

The remote server could not find the noarch directory for the
requested channel with url: https://conda.anaconda.org/cona-forge

As of conda 4.3, a valid channel must contain a `noarch/repodata.json` and
associated `noarch/repodata.json.bz2` file, even if `noarch/repodata.json` is
empty. please request that the channel administrator create
`noarch/repodata.json` and associated `noarch/repodata.json.bz2` files.
$ mkdir noarch
$ echo '{}' > noarch/repodata.json
$ bzip2 -k noarch/repodata.json

You will need to adjust your conda configuration to proceed.
Use `conda config --show channels` to view your configuration's current state.
Further configuration help can be found at <https://conda.io/docs/config.html>.




In [50]:
PATH = 'C:///Users/wchan/OneDrive/Documents/Learning_Coursera/IBM_data_science_professional/Geospatial_Coordinates.csv'

geo_df = pd.read_csv(PATH)

In [130]:
geo_df[:2]

PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497

In [52]:
# join with geospatial coordinates csv file to get latitude and longitude
final_df = pd.merge(df_borough_fil_grp_pc, geo_df,
                       how='left', on=['PostalCode'])
final_df[:11]

PostalCode      Borough                                       Neighborhood  \
0         M1B  Scarborough                                   [Rouge, Malvern]   
1         M1C  Scarborough           [Highland Creek, Rouge Hill, Port Union]   
2         M1E  Scarborough                [Guildwood, Morningside, West Hill]   
3         M1G  Scarborough                                           [Woburn]   
4         M1H  Scarborough                                        [Cedarbrae]   
5         M1J  Scarborough                              [Scarborough Village]   
6         M1K  Scarborough      [East Birchmount Park, Ionview, Kennedy Park]   
7         M1L  Scarborough                  [Clairlea, Golden Mile, Oakridge]   
8         M1M  Scarborough  [Cliffcrest, Cliffside, Scarborough Village West]   
9         M1N  Scarborough                      [Birch Cliff, Cliffside West]   
10        M1P  Scarborough  [Dorset Park, Scarborough Town Centre, Wexford...   

     Latitude  Longitude  
0   43.806686 -79.194353  
1   43.784535 -79.160497  
2   43.763573 -79.188711  
3   43.770992 -79.216917  
4   43.773136 -79.239476  
5   43.744734 -79.239476  
6   43.727929 -79.262029  
7   43.711112 -79.284577  
8   43.716316 -79.239476  
9   43.692657 -79.264848  
10  43.757410 -79.273304

### Explore and cluster the neighborhoods in Toronto

In [134]:
df_borough_fil = df_borough_fil.rename(columns={'Postcode': 'PostalCode'})

explore_df = pd.merge(df_borough_fil, geo_df,
                       how='left', on=['PostalCode'])
explore_df[:5]

PostalCode           Borough      Neighborhood   Latitude  Longitude
0        M3A        North York         Parkwoods  43.753259 -79.329656
1        M4A        North York  Victoria Village  43.725882 -79.315572
2        M5A  Downtown Toronto      Harbourfront  43.654260 -79.360636
3        M5A  Downtown Toronto       Regent Park  43.654260 -79.360636
4        M6A        North York  Lawrence Heights  43.718518 -79.464763

In [135]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [136]:
import folium # map rendering library

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

neighborhoods = explore_df

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [137]:
Scarborough_data = explore_df[explore_df['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarborough_data.head()

PostalCode      Borough    Neighborhood   Latitude  Longitude
0        M1B  Scarborough           Rouge  43.806686 -79.194353
1        M1B  Scarborough         Malvern  43.806686 -79.194353
2        M1C  Scarborough  Highland Creek  43.784535 -79.160497
3        M1C  Scarborough      Rouge Hill  43.784535 -79.160497
4        M1C  Scarborough      Port Union  43.784535 -79.160497

In [138]:
address = 'Scarborough, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [139]:
# create map of Manhattan using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'], Scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [140]:
CLIENT_ID = '3JUYIKWUE3BL2TSEDVNOE1VQS0LDEZDG1RTSCQPNAOFJRPQH' # your Foursquare ID
CLIENT_SECRET = 'CEYQWQKLZ1EK2DIM4MJGPO4JSDMMUEO02AWFDG53HTL1KM2Z' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3JUYIKWUE3BL2TSEDVNOE1VQS0LDEZDG1RTSCQPNAOFJRPQH
CLIENT_SECRET:CEYQWQKLZ1EK2DIM4MJGPO4JSDMMUEO02AWFDG53HTL1KM2Z


In [141]:
# CLIENT_ID = 'your-client-ID' # your Foursquare ID
# CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
# VERSION = '20180605' # Foursquare API version

# print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

In [142]:
Scarborough_data.loc[0, 'Neighborhood']

'Rouge'

In [143]:
neighborhood_latitude = Scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Scarborough_data.loc[0, 'Neighborhood']# neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge are 43.806686299999996, -79.19435340000001.


In [144]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=3JUYIKWUE3BL2TSEDVNOE1VQS0LDEZDG1RTSCQPNAOFJRPQH&client_secret=CEYQWQKLZ1EK2DIM4MJGPO4JSDMMUEO02AWFDG53HTL1KM2Z&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=1000&limit=100'

In [145]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ce1e623dd57970e24e6a915'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4d669cba83865481c948fa53-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/spa_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1ed941735',
         'name': 'Spa',
         'pluralName': 'Spas',
         'primary': True,
         'shortName': 'Spa'}],
       'id': '4d669cba83865481c948fa53',
       'location': {'address': '8130 Sheppard Ave E',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Morningside Ave',
        'distance': 595,
        'formattedAddress': ['8130 Sheppard Ave E (Morningside Ave)',
         'Toronto ON M1B 3W3',
         'Canada'],
        'labeledLatLngs': [{'label': 'd

In [146]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [147]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name                     categories        lat        lng
0   Images Salon & Spa                            Spa  43.802283 -79.198565
1       Caribbean Wave           Caribbean Restaurant  43.798558 -79.195777
2  Staples Morningside  Paper / Office Supplies Store  43.800285 -79.196607
3              Wendy's           Fast Food Restaurant  43.802008 -79.198080
4              Wendy's           Fast Food Restaurant  43.807448 -79.199056

In [148]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

17 venues were returned by Foursquare.


### Explore Neighborhoods in Manhattan

In [149]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [150]:
Scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighborhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )


Rouge
Malvern
Highland Creek
Rouge Hill
Port Union
Guildwood
Morningside
West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park
Ionview
Kennedy Park
Clairlea
Golden Mile
Oakridge
Cliffcrest
Cliffside
Scarborough Village West
Birch Cliff
Cliffside West
Dorset Park
Scarborough Town Centre
Wexford Heights
Maryvale
Wexford
Agincourt
Clarks Corners
Sullivan
Tam O'Shanter
Agincourt North
L'Amoreaux East
Milliken
Steeles East
L'Amoreaux West
Upper Rouge


In [151]:
print(Scarborough_venues.shape)
Scarborough_venues.head()

(891, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0        Rouge              43.806686              -79.194353   
1        Rouge              43.806686              -79.194353   
2        Rouge              43.806686              -79.194353   
3        Rouge              43.806686              -79.194353   
4        Rouge              43.806686              -79.194353   

                 Venue  Venue Latitude  Venue Longitude  \
0   Images Salon & Spa       43.802283       -79.198565   
1       Caribbean Wave       43.798558       -79.195777   
2  Staples Morningside       43.800285       -79.196607   
3              Wendy's       43.802008       -79.198080   
4              Wendy's       43.807448       -79.199056   

                  Venue Category  
0                            Spa  
1           Caribbean Restaurant  
2  Paper / Office Supplies Store  
3           Fast Food Restaurant  
4           Fast Food Restaurant

In [152]:
Scarborough_venues.groupby('Neighborhood').count()

Neighborhood Latitude  Neighborhood Longitude  \
Neighborhood                                                              
Agincourt                                    43                      43   
Agincourt North                              29                      29   
Birch Cliff                                  16                      16   
Cedarbrae                                    27                      27   
Clairlea                                     29                      29   
Clarks Corners                               37                      37   
Cliffcrest                                   12                      12   
Cliffside                                    12                      12   
Cliffside West                               16                      16   
Dorset Park                                  47                      47   
East Birchmount Park                         27                      27   
Golden Mile                                  29                      29   
Guildwood                                    22                      22   
Highland Creek                                5                       5   
Ionview                                      27                      27   
Kennedy Park                                 27                      27   
L'Amoreaux East                              29                      29   
L'Amoreaux West                              24                      24   
Malvern                                      17                      17   
Maryvale                                     30                      30   
Milliken                                     29                      29   
Morningside                                  22                      22   
Oakridge                                     29                      29   
Port Union                                    5                       5   
Rouge                                        17                      17   
Rouge Hill                                    5                       5   
Scarborough Town Centre                      47                      47   
Scarborough Village                          11                      11   
Scarborough Village West                     12                      12   
Steeles East                                 29                      29   
Sullivan                                     37                      37   
Tam O'Shanter                                37                      37   
West Hill                                    22                      22   
Wexford                                      30                      30   
Wexford Heights                              47                      47   
Woburn                                        7                       7   

                          Venue  Venue Latitude  Venue Longitude  \
Neighborhood                                                       
Agincourt                    43              43               43   
Agincourt North              29              29               29   
Birch Cliff                  16              16               16   
Cedarbrae                    27              27               27   
Clairlea                     29              29               29   
Clarks Corners               37              37               37   
Cliffcrest                   12              12               12   
Cliffside                    12              12               12   
Cliffside West               16              16               16   
Dorset Park                  47              47               47   
East Birchmount Park         27              27               27   
Golden Mile                  29              29               29   
Guildwood                    22              22               22   
Highland Creek                5               5                5   
Ionview                      27              27               27   
Kennedy Park       

In [153]:
print('There are {} uniques categories.'.format(len(Scarborough_venues['Venue Category'].unique())))

There are 108 uniques categories.


### Analyze Each Neighborhood

In [154]:
# one hot encoding
scar_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scar_onehot['Neighborhood'] = Scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scar_onehot.columns[-1]] + list(scar_onehot.columns[:-1])
scar_onehot = scar_onehot[fixed_columns]

scar_onehot.head()

Neighborhood  African Restaurant  American Restaurant  Asian Restaurant  \
0        Rouge                   0                    0                 0   
1        Rouge                   0                    0                 0   
2        Rouge                   0                    0                 0   
3        Rouge                   0                    0                 0   
4        Rouge                   0                    0                 0   

   Athletics & Sports  Auto Garage  Automotive Shop  BBQ Joint  \
0                   0            0                0          0   
1                   0            0                0          0   
2                   0            0                0          0   
3                   0            0                0          0   
4                   0            0                0          0   

   Badminton Court  Bakery     ...       Taiwanese Restaurant  Tennis Court  \
0                0       0     ...                          0             0   
1                0       0     ...                          0             0   
2                0       0     ...                          0             0   
3                0       0     ...                          0             0   
4                0       0     ...                          0             0   

   Thai Restaurant  Trail  Train Station  Vegetarian / Vegan Restaurant  \
0                0      0              0                              0   
1                0      0              0                              0   
2                0      0              0                              0   
3                0      0              0                              0   
4                0      0              0                              0   

   Video Game Store  Vietnamese Restaurant  Wings Joint  Yoga Studio  
0                 0                      0            0            0  
1                 0                      0            0            0  
2                 0                      0            0            0  
3                 0                      0            0            0  
4                 0                      0            0            0  

[5 rows x 109 columns]

In [155]:
scar_onehot.shape

(891, 109)

In [156]:
scar_grouped = scar_onehot.groupby('Neighborhood').mean().reset_index()
scar_grouped[:5]

Neighborhood  African Restaurant  American Restaurant  Asian Restaurant  \
0        Agincourt                 0.0             0.023256            0.0000   
1  Agincourt North                 0.0             0.000000            0.0000   
2      Birch Cliff                 0.0             0.000000            0.0625   
3        Cedarbrae                 0.0             0.000000            0.0000   
4         Clairlea                 0.0             0.000000            0.0000   

   Athletics & Sports  Auto Garage  Automotive Shop  BBQ Joint  \
0            0.000000       0.0000              0.0   0.023256   
1            0.000000       0.0000              0.0   0.000000   
2            0.000000       0.0625              0.0   0.000000   
3            0.037037       0.0000              0.0   0.000000   
4            0.000000       0.0000              0.0   0.000000   

   Badminton Court    Bakery     ...       Taiwanese Restaurant  Tennis Court  \
0              0.0  0.046512     ...                        0.0           0.0   
1              0.0  0.068966     ...                        0.0           0.0   
2              0.0  0.000000     ...                        0.0           0.0   
3              0.0  0.111111     ...                        0.0           0.0   
4              0.0  0.068966     ...                        0.0           0.0   

   Thai Restaurant     Trail  Train Station  Vegetarian / Vegan Restaurant  \
0         0.000000  0.000000            0.0                       0.000000   
1         0.000000  0.000000            0.0                       0.034483   
2         0.062500  0.000000            0.0                       0.000000   
3         0.037037  0.000000            0.0                       0.000000   
4         0.000000  0.034483            0.0                       0.000000   

   Video Game Store  Vietnamese Restaurant  Wings Joint  Yoga Studio  
0               0.0               0.023256     0.000000     0.000000  
1               0.0               0.000000     0.000000     0.000000  
2               0.0               0.000000     0.000000     0.000000  
3               0.0               0.000000     0.037037     0.037037  
4               0.0               0.000000     0.000000     0.000000  

[5 rows x 109 columns]

In [157]:
num_top_venues = 5

for hood in scar_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scar_grouped[scar_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.12
1         Shopping Mall  0.09
2           Supermarket  0.05
3  Caribbean Restaurant  0.05
4        Sandwich Place  0.05


----Agincourt North----
                venue  freq
0  Chinese Restaurant  0.24
1         Pizza Place  0.07
2                Park  0.07
3              Bakery  0.07
4        Noodle House  0.07


----Birch Cliff----
                   venue  freq
0           Dessert Shop  0.06
1                   Bank  0.06
2  General Entertainment  0.06
3                   Café  0.06
4                    Gym  0.06


----Cedarbrae----
               venue  freq
0           Pharmacy  0.11
1             Bakery  0.11
2        Coffee Shop  0.11
3  Indian Restaurant  0.07
4           Bus Line  0.04


----Clairlea----
                  venue  freq
0          Intersection  0.10
1           Coffee Shop  0.10
2  Fast Food Restaurant  0.07
3              Bus Line  0.07
4                Bakery  0.07


----Clarks Corner

In [158]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [159]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scar_grouped['Neighborhood']

for ind in np.arange(scar_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scar_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0        Agincourt    Chinese Restaurant         Shopping Mall   
1  Agincourt North    Chinese Restaurant          Noodle House   
2      Birch Cliff                  Park          Dessert Shop   
3        Cedarbrae           Coffee Shop              Pharmacy   
4         Clairlea          Intersection           Coffee Shop   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                Bakery  Caribbean Restaurant           Pizza Place   
1                  Park           Pizza Place                Bakery   
2                  Café          Skating Rink            Restaurant   
3                Bakery     Indian Restaurant          Burger Joint   
4                  Park              Bus Line  Fast Food Restaurant   

  6th Most Common Venue  7th Most Common Venue 8th Most Common Venue  \
0        Sandwich Place            Supermarket            Restaurant   
1                   Spa            Coffee Shop              Pharmacy   
2        Discount Store  General Entertainment                  Bank   
3           Wings Joint          Grocery Store   Fried Chicken Joint   
4                Bakery      Convenience Store            Beer Store   

  9th Most Common Venue 10th Most Common Venue  
0       Bubble Tea Shop              Pool Hall  
1  Caribbean Restaurant           Dessert Shop  
2                 Diner                    Gym  
3  Fast Food Restaurant            Music Store  
4        Sandwich Place            Bus Station

### Cluster Neighborhoods

In [160]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

scar_grouped_clustering = scar_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scar_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 0, 0, 0, 3, 3, 0, 0])

In [161]:
Scarborough_data[:2]

PostalCode      Borough Neighborhood   Latitude  Longitude
0        M1B  Scarborough        Rouge  43.806686 -79.194353
1        M1B  Scarborough      Malvern  43.806686 -79.194353

In [162]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scar_merged = Scarborough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scar_merged = scar_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scar_merged.head() # check the last columns!

PostalCode      Borough    Neighborhood   Latitude  Longitude  \
0        M1B  Scarborough           Rouge  43.806686 -79.194353   
1        M1B  Scarborough         Malvern  43.806686 -79.194353   
2        M1C  Scarborough  Highland Creek  43.784535 -79.160497   
3        M1C  Scarborough      Rouge Hill  43.784535 -79.160497   
4        M1C  Scarborough      Port Union  43.784535 -79.160497   

   Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0             0.0  Fast Food Restaurant    African Restaurant   
1             0.0  Fast Food Restaurant    African Restaurant   
2             2.0        Breakfast Spot    Italian Restaurant   
3             2.0        Breakfast Spot    Italian Restaurant   
4             2.0        Breakfast Spot    Italian Restaurant   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                   Spa                   Gym      Greek Restaurant   
1                   Spa                   Gym      Greek Restaurant   
2                  Park          Burger Joint            Playground   
3                  Park          Burger Joint            Playground   
4                  Park          Burger Joint            Playground   

     6th Most Common Venue          7th Most Common Venue  \
0  Fruit & Vegetable Store  Paper / Office Supplies Store   
1  Fruit & Vegetable Store  Paper / Office Supplies Store   
2              Yoga Studio               Department Store   
3              Yoga Studio               Department Store   
4              Yoga Studio               Department Store   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0                  Park    Chinese Restaurant   Caribbean Restaurant  
1                  Park    Chinese Restaurant   Caribbean Restaurant  
2          Dessert Shop                 Diner         Discount Store  
3          Dessert Shop                 Diner         Discount Store  
4          Dessert Shop                 Diner         Discount Store

In [172]:
scar_merged['Cluster Labels'] = scar_merged['Cluster Labels'].fillna(0).astype(int)
scar_merged.head()

PostalCode      Borough    Neighborhood   Latitude  Longitude  \
0        M1B  Scarborough           Rouge  43.806686 -79.194353   
1        M1B  Scarborough         Malvern  43.806686 -79.194353   
2        M1C  Scarborough  Highland Creek  43.784535 -79.160497   
3        M1C  Scarborough      Rouge Hill  43.784535 -79.160497   
4        M1C  Scarborough      Port Union  43.784535 -79.160497   

   Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0               0  Fast Food Restaurant    African Restaurant   
1               0  Fast Food Restaurant    African Restaurant   
2               2        Breakfast Spot    Italian Restaurant   
3               2        Breakfast Spot    Italian Restaurant   
4               2        Breakfast Spot    Italian Restaurant   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                   Spa                   Gym      Greek Restaurant   
1                   Spa                   Gym      Greek Restaurant   
2                  Park          Burger Joint            Playground   
3                  Park          Burger Joint            Playground   
4                  Park          Burger Joint            Playground   

     6th Most Common Venue          7th Most Common Venue  \
0  Fruit & Vegetable Store  Paper / Office Supplies Store   
1  Fruit & Vegetable Store  Paper / Office Supplies Store   
2              Yoga Studio               Department Store   
3              Yoga Studio               Department Store   
4              Yoga Studio               Department Store   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0                  Park    Chinese Restaurant   Caribbean Restaurant  
1                  Park    Chinese Restaurant   Caribbean Restaurant  
2          Dessert Shop                 Diner         Discount Store  
3          Dessert Shop                 Diner         Discount Store  
4          Dessert Shop                 Diner         Discount Store

In [173]:

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scar_merged['Latitude'], scar_merged['Longitude'], scar_merged['Neighborhood'], scar_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

In [174]:
scar_merged.loc[scar_merged['Cluster Labels'] == 0, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

Borough  Cluster Labels      1st Most Common Venue  \
0   Scarborough               0       Fast Food Restaurant   
1   Scarborough               0       Fast Food Restaurant   
8   Scarborough               0                Coffee Shop   
9   Scarborough               0                Coffee Shop   
11  Scarborough               0             Discount Store   
12  Scarborough               0             Discount Store   
13  Scarborough               0             Discount Store   
14  Scarborough               0               Intersection   
15  Scarborough               0               Intersection   
16  Scarborough               0               Intersection   
20  Scarborough               0                       Park   
21  Scarborough               0                       Park   
22  Scarborough               0                Coffee Shop   
23  Scarborough               0                Coffee Shop   
24  Scarborough               0                Coffee Shop   
25  Scarborough               0  Middle Eastern Restaurant   
26  Scarborough               0  Middle Eastern Restaurant   
28  Scarborough               0                   Pharmacy   
29  Scarborough               0                   Pharmacy   
30  Scarborough               0                   Pharmacy   
36  Scarborough               0                        NaN   

     2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0       African Restaurant                   Spa                   Gym   
1       African Restaurant                   Spa                   Gym   
8        Indian Restaurant    Chinese Restaurant                  Park   
9                 Pharmacy                Bakery     Indian Restaurant   
11             Coffee Shop    Chinese Restaurant         Grocery Store   
12             Coffee Shop    Chinese Restaurant         Grocery Store   
13             Coffee Shop    Chinese Restaurant         Grocery Store   
14             Coffee Shop                  Park              Bus Line   
15             Coffee Shop                  Park              Bus Line   
16             Coffee Shop                  Park              Bus Line   
20            Dessert Shop                  Café          Skating Rink   
21            Dessert Shop                  Café          Skating Rink   
22  Furniture / Home Store     Electronics Store      Asian Restaurant   
23  Furniture / Home Store     Electronics Store      Asian Restaurant   
24  Furniture / Home Store     Electronics Store      Asian Restaurant   
25             Pizza Place         Grocery Store          Burger Joint   
26             Pizza Place         Grocery Store          Burger Joint   
28             Pizza Place         Shopping Mall          Intersection   
29             Pizza Place         Shopping Mall          Intersection   
30             Pizza Place         Shopping Mall          Intersection   
36                     NaN                   NaN                   NaN   

   5th Most Common Venue    6th Most Common Venue  \
0       Greek Restaurant  Fruit & Vegetable Store   
1       Greek Restaurant  Fruit & Vegetable Store   
8      Electronics Store     Fast Food Restaurant   
9           Burger Joint              Wings Joint   
11  Fast Food Restaurant             Burger Joint   
12  Fast Food Restaurant             Burger Joint   
13  Fast Food Restaurant             Burger Joint   
14  Fast Food Restaurant                   Bakery   
15  Fast Food Restaurant                   Bakery   
16  Fast Food Restaurant                   Bakery   
20            Restaurant           Discount Store   
21            Restaurant           Discount Store   
22    Chinese Restaurant     Fast Food Restaurant   
23    Chinese Restaurant     Fast Food Restaurant   
24    Chinese Restaurant     Fast Food Restaurant   
25        Breakfast Spot              Gas Station   
26        Breakfast Spot              Gas Station   
28  Fast Food Restaurant                     Park   
29  Fas

In [175]:
scar_merged.loc[scar_merged['Cluster Labels'] == 1, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
27  Scarborough               1    Chinese Restaurant         Shopping Mall   
31  Scarborough               1    Chinese Restaurant          Noodle House   
32  Scarborough               1    Chinese Restaurant          Noodle House   
33  Scarborough               1    Chinese Restaurant          Noodle House   
34  Scarborough               1    Chinese Restaurant          Noodle House   
35  Scarborough               1    Chinese Restaurant           Coffee Shop   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
27                Bakery  Caribbean Restaurant           Pizza Place   
31                  Park           Pizza Place                Bakery   
32                  Park           Pizza Place                Bakery   
33                  Park           Pizza Place                Bakery   
34                  Park           Pizza Place                Bakery   
35  Fast Food Restaurant                Bakery        Breakfast Spot   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
27        Sandwich Place           Supermarket            Restaurant   
31                   Spa           Coffee Shop              Pharmacy   
32                   Spa           Coffee Shop              Pharmacy   
33                   Spa           Coffee Shop              Pharmacy   
34                   Spa           Coffee Shop              Pharmacy   
35     Hotpot Restaurant                   Gym         Grocery Store   

   9th Most Common Venue 10th Most Common Venue  
27       Bubble Tea Shop              Pool Hall  
31  Caribbean Restaurant           Dessert Shop  
32  Caribbean Restaurant           Dessert Shop  
33  Caribbean Restaurant           Dessert Shop  
34  Caribbean Restaurant           Dessert Shop  
35          Intersection   Other Great Outdoors

In [176]:
scar_merged.loc[scar_merged['Cluster Labels'] == 2, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
2  Scarborough               2        Breakfast Spot    Italian Restaurant   
3  Scarborough               2        Breakfast Spot    Italian Restaurant   
4  Scarborough               2        Breakfast Spot    Italian Restaurant   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
2                  Park          Burger Joint            Playground   
3                  Park          Burger Joint            Playground   
4                  Park          Burger Joint            Playground   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
2           Yoga Studio      Department Store          Dessert Shop   
3           Yoga Studio      Department Store          Dessert Shop   
4           Yoga Studio      Department Store          Dessert Shop   

  9th Most Common Venue 10th Most Common Venue  
2                 Diner         Discount Store  
3                 Diner         Discount Store  
4                 Diner         Discount Store

In [177]:
scar_merged.loc[scar_merged['Cluster Labels'] == 3, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
17  Scarborough               3           Pizza Place  Fast Food Restaurant   
18  Scarborough               3           Pizza Place  Fast Food Restaurant   
19  Scarborough               3           Pizza Place  Fast Food Restaurant   

   3rd Most Common Venue 4th Most Common Venue      5th Most Common Venue  \
17                 Beach                  Park  Cajun / Creole Restaurant   
18                 Beach                  Park  Cajun / Creole Restaurant   
19                 Beach                  Park  Cajun / Creole Restaurant   

     6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
17  Furniture / Home Store            Sports Bar          Burger Joint   
18  Furniture / Home Store            Sports Bar          Burger Joint   
19  Furniture / Home Store            Sports Bar          Burger Joint   

   9th Most Common Venue 10th Most Common Venue  
17           Yoga Studio            Fish Market  
18           Yoga Studio            Fish Market  
19           Yoga Studio            Fish Market

In [178]:
scar_merged.loc[scar_merged['Cluster Labels'] == 0, scar_merged.columns[[4] + list(range(5, scar_merged.shape[1]))]]

Longitude  Cluster Labels      1st Most Common Venue  \
0  -79.194353               0       Fast Food Restaurant   
1  -79.194353               0       Fast Food Restaurant   
8  -79.216917               0                Coffee Shop   
9  -79.239476               0                Coffee Shop   
11 -79.262029               0             Discount Store   
12 -79.262029               0             Discount Store   
13 -79.262029               0             Discount Store   
14 -79.284577               0               Intersection   
15 -79.284577               0               Intersection   
16 -79.284577               0               Intersection   
20 -79.264848               0                       Park   
21 -79.264848               0                       Park   
22 -79.273304               0                Coffee Shop   
23 -79.273304               0                Coffee Shop   
24 -79.273304               0                Coffee Shop   
25 -79.295849               0  Middle Eastern Restaurant   
26 -79.295849               0  Middle Eastern Restaurant   
28 -79.304302               0                   Pharmacy   
29 -79.304302               0                   Pharmacy   
30 -79.304302               0                   Pharmacy   
36 -79.205636               0                        NaN   

     2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0       African Restaurant                   Spa                   Gym   
1       African Restaurant                   Spa                   Gym   
8        Indian Restaurant    Chinese Restaurant                  Park   
9                 Pharmacy                Bakery     Indian Restaurant   
11             Coffee Shop    Chinese Restaurant         Grocery Store   
12             Coffee Shop    Chinese Restaurant         Grocery Store   
13             Coffee Shop    Chinese Restaurant         Grocery Store   
14             Coffee Shop                  Park              Bus Line   
15             Coffee Shop                  Park              Bus Line   
16             Coffee Shop                  Park              Bus Line   
20            Dessert Shop                  Café          Skating Rink   
21            Dessert Shop                  Café          Skating Rink   
22  Furniture / Home Store     Electronics Store      Asian Restaurant   
23  Furniture / Home Store     Electronics Store      Asian Restaurant   
24  Furniture / Home Store     Electronics Store      Asian Restaurant   
25             Pizza Place         Grocery Store          Burger Joint   
26             Pizza Place         Grocery Store          Burger Joint   
28             Pizza Place         Shopping Mall          Intersection   
29             Pizza Place         Shopping Mall          Intersection   
30             Pizza Place         Shopping Mall          Intersection   
36                     NaN                   NaN                   NaN   

   5th Most Common Venue    6th Most Common Venue  \
0       Greek Restaurant  Fruit & Vegetable Store   
1       Greek Restaurant  Fruit & Vegetable Store   
8      Electronics Store     Fast Food Restaurant   
9           Burger Joint              Wings Joint   
11  Fast Food Restaurant             Burger Joint   
12  Fast Food Restaurant             Burger Joint   
13  Fast Food Restaurant             Burger Joint   
14  Fast Food Restaurant                   Bakery   
15  Fast Food Restaurant                   Bakery   
16  Fast Food Restaurant                   Bakery   
20            Restaurant           Discount Store   
21            Restaurant           Discount Store   
22    Chinese Restaurant     Fast Food Restaurant   
23    Chinese Restaurant     Fast Food Restaurant   
24    Chinese Restaurant     Fast Food Restaurant   
25        Breakfast Spot              Gas Station   
26        Breakfast Spot              Gas Station   
28  Fast Food Restaurant                     Park   
29  Fast Food Restaurant                     Pa

In [179]:
scar_merged.loc[scar_merged['Cluster Labels'] == 0, scar_merged.columns[[5] + list(range(5, scar_merged.shape[1]))]]

Cluster Labels  Cluster Labels      1st Most Common Venue  \
0                0               0       Fast Food Restaurant   
1                0               0       Fast Food Restaurant   
8                0               0                Coffee Shop   
9                0               0                Coffee Shop   
11               0               0             Discount Store   
12               0               0             Discount Store   
13               0               0             Discount Store   
14               0               0               Intersection   
15               0               0               Intersection   
16               0               0               Intersection   
20               0               0                       Park   
21               0               0                       Park   
22               0               0                Coffee Shop   
23               0               0                Coffee Shop   
24               0               0                Coffee Shop   
25               0               0  Middle Eastern Restaurant   
26               0               0  Middle Eastern Restaurant   
28               0               0                   Pharmacy   
29               0               0                   Pharmacy   
30               0               0                   Pharmacy   
36               0               0                        NaN   

     2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0       African Restaurant                   Spa                   Gym   
1       African Restaurant                   Spa                   Gym   
8        Indian Restaurant    Chinese Restaurant                  Park   
9                 Pharmacy                Bakery     Indian Restaurant   
11             Coffee Shop    Chinese Restaurant         Grocery Store   
12             Coffee Shop    Chinese Restaurant         Grocery Store   
13             Coffee Shop    Chinese Restaurant         Grocery Store   
14             Coffee Shop                  Park              Bus Line   
15             Coffee Shop                  Park              Bus Line   
16             Coffee Shop                  Park              Bus Line   
20            Dessert Shop                  Café          Skating Rink   
21            Dessert Shop                  Café          Skating Rink   
22  Furniture / Home Store     Electronics Store      Asian Restaurant   
23  Furniture / Home Store     Electronics Store      Asian Restaurant   
24  Furniture / Home Store     Electronics Store      Asian Restaurant   
25             Pizza Place         Grocery Store          Burger Joint   
26             Pizza Place         Grocery Store          Burger Joint   
28             Pizza Place         Shopping Mall          Intersection   
29             Pizza Place         Shopping Mall          Intersection   
30             Pizza Place         Shopping Mall          Intersection   
36                     NaN                   NaN                   NaN   

   5th Most Common Venue    6th Most Common Venue  \
0       Greek Restaurant  Fruit & Vegetable Store   
1       Greek Restaurant  Fruit & Vegetable Store   
8      Electronics Store     Fast Food Restaurant   
9           Burger Joint              Wings Joint   
11  Fast Food Restaurant             Burger Joint   
12  Fast Food Restaurant             Burger Joint   
13  Fast Food Restaurant             Burger Joint   
14  Fast Food Restaurant                   Bakery   
15  Fast Food Restaurant                   Bakery   
16  Fast Food Restaurant                   Bakery   
20            Restaurant           Discount Store   
21            Restaurant           Discount Store   
22    Chinese Restaurant     Fast Food Restaurant   
23    Chinese Restaurant     Fast Food Restaurant   
24    Chinese Restaurant     Fast Food Restaurant   
25        Breakfast Spot              Gas Station   
26        Breakfast Spot              Gas S